In [1]:
!git clone https://github.com/RizwanMunawar/yolov7-pose-estimation.git

Cloning into 'yolov7-pose-estimation'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 193 (delta 42), reused 32 (delta 32), pack-reused 138
Receiving objects: 100% (193/193), 3.75 MiB | 2.19 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [2]:
cd yolov7-pose-estimation

/content/yolov7-pose-estimation


In [3]:
!pip install --upgrade pip
!pip install -r requirements.txt
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 731.7/731.7 MB 177.5 MB/s eta 0:00:01
ERROR: Operation cancelled by user
--2024-07-15 05:07:02--  https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-productio

In [ ]:
#REMEMBER TO REPLACE pose-estimate.py
#Refer to replacement code in pose_estimation.ipynb depending on whether you want to do inference on images or videos.

In [7]:
cd /content

/content


In [4]:
!pip install ultralytics

!pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from segment_anything import SamPredictor, sam_model_registry
from PIL import Image


### SEGMENTATION CODE FOR IMAGE

In [8]:
model = YOLO('/content/yolov8n.pt')

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels == 1]
    neg_points = coords[labels == 0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))

def remove_background(image_path, output_path, yolo_model, sam_model, sam_predictor):
    # Predict using YOLOv8
    results = yolo_model.predict(source=image_path, conf=0.25)

    for result in results:
        boxes = result.boxes
        if len(boxes) > 0:
            bbox = boxes.xyxy[0].cpu().numpy()
            break
    else:
        print("No person detected in the image.")
        return


    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    # Predict using SAM
    sam_predictor.set_image(image)
    input_box = np.array(bbox)

    masks, _, _ = sam_predictor.predict(
        point_coords=None,
        point_labels=None,
        box=input_box[None, :],
        multimask_output=False,
    )

    segmentation_mask = masks[0]
    binary_mask = np.where(segmentation_mask > 0.5, 1, 0)
    white_background = np.ones_like(image) * 255
    new_image = white_background * (1 - binary_mask[..., np.newaxis]) + image * binary_mask[..., np.newaxis]

    result_image = Image.fromarray(new_image.astype(np.uint8))
    result_image.save(output_path)
    print(f"Background removed image saved to {output_path}")

sam_checkpoint = "/content/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
predictor = SamPredictor(sam)


In [ ]:
import os
#replace image image with your own image
image_name = 'walk9.jpg'
image_path = f'/content/{image_name}'
img_name = image_name.split('.')[0]
output_path = f'/content/{img_name}_bg_removed.png'


remove_background(image_path, output_path, model, sam, predictor)

### SEGMENTATION CODE FOR VIDEO

In [28]:
import cv2
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from segment_anything import SamPredictor, sam_model_registry
from ultralytics import YOLO


def extract_frames(video_path, frames_dir):
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    video_capture = cv2.VideoCapture(video_path)
    frame_rate = video_capture.get(cv2.CAP_PROP_FPS) 
    frame_count = 0

    while True:
        success, frame = video_capture.read()
        if not success:
            break
        frame_path = os.path.join(frames_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    video_capture.release()
    return frame_count, frame_rate

def remove_background_frame(frame_path, processed_frames_dir, yolo_model, sam_predictor):
    frame = cv2.imread(frame_path)
    results = yolo_model.predict(source=frame, conf=0.25, device='cuda')
    for result in results:
        boxes = result.boxes
        if len(boxes) > 0:
            bbox = boxes.xyxy[0].cpu().numpy()
            break
    else:
        return  

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    sam_predictor.set_image(image)
    input_box = np.array(bbox)
    masks, _, _ = sam_predictor.predict(
        point_coords=None,
        point_labels=None,
        box=input_box[None, :],
        multimask_output=False,
    )

    segmentation_mask = masks[0]
    binary_mask = np.where(segmentation_mask > 0.5, 1, 0)
    white_background = np.ones_like(image) * 255
    new_image = white_background * (1 - binary_mask[..., np.newaxis]) + image * binary_mask[..., np.newaxis]

    processed_frame = cv2.cvtColor(new_image.astype(np.uint8), cv2.COLOR_RGB2BGR)
    processed_frame_path = os.path.join(processed_frames_dir, os.path.basename(frame_path))
    cv2.imwrite(processed_frame_path, processed_frame)

def process_frames_with_progress_bar(frame_paths, processed_frames_dir, yolo_model, sam_predictor):
    for frame_path in tqdm(frame_paths, desc="Processing frames"):
        remove_background_frame(frame_path, processed_frames_dir, yolo_model, sam_predictor)

def create_video_from_frames(frames_dir, output_video_path, frame_rate):
    frames = [f for f in sorted(os.listdir(frames_dir)) if f.endswith(".jpg")]
    frame_example = cv2.imread(os.path.join(frames_dir, frames[0]))
    height, width, layers = frame_example.shape

    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (width, height))

    for frame_name in frames:
        frame_path = os.path.join(frames_dir, frame_name)
        frame = cv2.imread(frame_path)
        video_writer.write(frame)

    video_writer.release()



In [ ]:
#replace with your own vid name
vid_name = 'walk2.mp4'
video_path = f'/content/{vid_name}'
original_frames_dir = '/content/original_frames'
processed_frames_dir = '/content/processed_frames'
name = vid_name.split('.')[0]
output_video_path = f'/content/{name}_bg_removed.mp4'
sam_checkpoint = "/content/sam_vit_h_4b8939.pth"
model_path = "/content/yolov8n.pt"

yolo_model = YOLO(model_path)

model_type = "vit_h"
sam_model = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam_predictor = SamPredictor(sam_model)

frame_count, frame_rate = extract_frames(video_path, original_frames_dir)

if not os.path.exists(processed_frames_dir):
    os.makedirs(processed_frames_dir)

frame_paths = [os.path.join(original_frames_dir, f"frame_{i:04d}.jpg") for i in range(frame_count)]
process_frames_with_progress_bar(frame_paths, processed_frames_dir, yolo_model, sam_predictor)

In [31]:
create_video_from_frames(processed_frames_dir, output_video_path, frame_rate)

In [15]:
cd /content/yolov7-pose-estimation

/content/yolov7-pose-estimation


In [34]:
output_vid_name = 'walk2_bg_removed.mp4'

In [35]:

# run pose estimation using the img/video with background removed.
cmd = f"python pose-estimate.py --source {output_vid_name} --device 0"
os.system(cmd)

0